# word2vec implementation

## Overview

The original paper proposed using:
1. Matmul to extract word vectors from ```Win```.
2. Softmax to calculate scores with all the word vectors from ```Wout```.

<img src="image/word2vec_cbow_mechanism.png" align="left"/>

## Negative Sampling

### Motivation

The original implementation was computationally expensive.

1. Matmal to extract word vectors from ```Win```.
2. Softmax can happen vocabulary size times with ```Wout```.

<img src="image/word2vec_negative_sampling_motivation.png" align="left"/>


### Solution

1. Use index to extract word vectors from Win.
2. Instead of calculating softmax with all the word vectors from ```Wout```, sample small number (SL) of negative/false word vectors from ```Wout``` and calculate logistic log loss with each sample. 

<img src="image/wors2vec_neg_sample_backprop.png" align="left"/>

## Using only one Word vector space W

There is no reasoning nor proof why two word vector space are required. In the end, we only use one word vector space, which appears to be ```Win```. Use only one word vector space ```W``` to test if it works.

<img src="image/word2vec_backprop_Be.png" align="left"/>

---
# Setups

In [1]:
import cProfile
import sys
import os
import re
from itertools import islice
from typing import Dict, List
import numpy as np
import tensorflow as tf

np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(linewidth=400) 

## Setup for Google Colab environment

### Clone github to Google Drive

In [2]:
try:
    import google.colab
    IN_GOOGLE_COLAB = True
except:
    IN_GOOGLE_COLAB = False
    
if IN_GOOGLE_COLAB:
    !pip install line_profiler
    !google.colab.drive.mount('/content/gdrive')
    !rm -rf /content/drive/MyDrive/github
    !mkdir -p /content/drive/MyDrive/github
    !git clone https://github.com/oonisim/python-programs.git /content/drive/MyDrive/github


### Clone github to local directory

In [3]:
try:
    import google.colab
    IN_GOOGLE_COLAB = True
except:
    IN_GOOGLE_COLAB = False
    
if IN_GOOGLE_COLAB:
    !pip install line_profiler
    !google.colab.drive.mount('/content/gdrive')
    !rm -rf /content/github
    !mkdir -p /content/github
    !git clone https://github.com/oonisim/python-programs.git /content/github
        
    import sys
    sys.path.append('/content/github/nlp/src')

## Jupyter notebook setups

In [4]:
%load_ext line_profiler
%load_ext autoreload

## Utilites

In [35]:
%autoreload 2

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

import function.fileio as fileio
import function.text as text

---
# Data Types


In [6]:
from common.constant import (
    TYPE_INT,
    TYPE_FLOAT,
    TYPE_LABEL,
    TYPE_TENSOR,
)

# Constants

In [7]:
USE_PTB = True
DEBUG = False
VALIDATION = True

TARGET_SIZE = 1   # Size of the target event (word)
CONTEXT_SIZE = 4  # Size of the context.
WINDOW_SIZE = TARGET_SIZE + CONTEXT_SIZE
SAMPLE_SIZE = 5   # Size of the negative samples
VECTOR_SIZE = 30  # Number of features in the event vector.

---

# Data
## Corpus

In [8]:
corpus = "To be, or not to be, that is the question that matters"
_file = "ptb.train.txt"
if USE_PTB:
    if not fileio.Function.is_file(f"~/.keras/datasets/{_file}"):
        path_to_ptb = tf.keras.utils.get_file(
            _file, 
            f'https://raw.githubusercontent.com/tomsercu/lstm/master/data/{_file}'
        )
    corpus = fileio.Function.read_file(path_to_ptb)

In [9]:
examples = corpus.split('\n')[:5]
for line in examples:
    print(line)

 aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memotec mlx nahb punts rake regatta rubens sim snack-food ssangyong swapo wachter 
 pierre <unk> N years old will join the board as a nonexecutive director nov. N 
 mr. <unk> is chairman of <unk> n.v. the dutch publishing group 
 rudolph <unk> N years old and former chairman of consolidated gold fields plc was named a nonexecutive director of this british industrial conglomerate 
 a form of asbestos once used to make kent cigarette filters has caused a high percentage of cancer deaths among a group of workers exposed to it more than N years ago researchers reported 


---
# Event (word) indexing
Index the events that have occurred in the event sequence.

In [10]:
%autoreload 2
from layer.preprocessing import (
    EventIndexing, 
)

In [11]:
word_indexing = EventIndexing(
    name="word_indexing_on_ptb",
    corpus=corpus
)
del corpus

## EventIndexing  for the corpus

Adapt to the ```corpus``` and provides:
* event_to_index dictionary
* vocaburary of the corpus
* word occurrence probabilites

In [12]:
words = word_indexing.list_events(range(10))
print(f"EventIndexing.vocabulary[10]:\n{words}\n")

indices = word_indexing.list_indices(words)
print(f"EventIndexing.event_to_index[10]:")
for item in zip(words, indices):
    print(item)

probabilities = word_indexing.list_probabilities(words)
print(f"\nEventIndexing.probabilities[10]:")
for word, p in zip(words, probabilities):
    print(f"{word:20s} : {p:.5e}")

EventIndexing.vocabulary[10]:
['<nil>' '<unk>' 'aer' 'banknote' 'berlitz' 'calloway' 'centrust' 'cluett' 'fromstein' 'gitano']

EventIndexing.event_to_index[10]:
('<nil>', 0)
('<unk>', 1)
('aer', 2)
('banknote', 3)
('berlitz', 4)
('calloway', 5)
('centrust', 6)
('cluett', 7)
('fromstein', 8)
('gitano', 9)

EventIndexing.probabilities[10]:
<nil>                : 0.00000e+00
<unk>                : 1.65308e-02
aer                  : 5.34860e-06
banknote             : 5.34860e-06
berlitz              : 5.34860e-06
calloway             : 5.34860e-06
centrust             : 5.34860e-06
cluett               : 5.34860e-06
fromstein            : 5.34860e-06
gitano               : 5.34860e-06


## Sampling using the probability

Sample events according to their probabilities.

In [13]:
sample = word_indexing.sample(size=5)
print(sample)

['dollar', 'bank', 'nobody', 'salvage', 'people']


## Negative Sampling
Sample events not including those events already sampled.

In [14]:
negative_indices = word_indexing.negative_sample_indices(
    size=5, excludes=word_indexing.list_indices(sample)
)
print(f"negative_indices={negative_indices} \nevents={word_indexing.list_events(negative_indices)}")


negative_indices=[9024, 1194, 236, 466, 50] 
events=['grave' 'rather' 'support' 'or' 'and']


## Sentence to Sequence

In [15]:
# sentences = "\n".join(corpus.split('\n')[5:6])
sentences = """
the asbestos fiber <unk> is unusually <unk> once it enters the <unk> 
with even brief exposures to it causing symptoms that show up decades later researchers said
"""
sequences = word_indexing.function(sentences)
for pair in zip(sentences.strip().split(" "), sequences[0]):
    print(f"{pair[0]:15} : {pair[1]:5}")

Sentence is empty. Skipping...
Sentence is empty. Skipping...


the             :    34
asbestos        :    63
fiber           :    86
<unk>           :     1
is              :    42
unusually       :    87
<unk>           :     1
once            :    64
it              :    80
enters          :    88
the             :    34
<unk>           :     1

with           :     0
even            :     0
brief           :     0


---
# EventContext


In [16]:
%autoreload 2
from layer.preprocessing import (
    EventContext
)

In [17]:
event_context = EventContext(
    name="ev",
    window_size=WINDOW_SIZE,
    event_size=TARGET_SIZE
)

## Context of an event (word) in a sentence

In the sentence ```"a form of asbestos once used to make kent cigarette filters"```, one of the context windows ```a form of asbestos once``` of size 5 and event size 1 has.
* ```of``` as a target word.
* ```(a, form) and (asbestos, once)``` as its context.

### Sequence of the word indices for the sentence

In [18]:
sentences = """
a form of asbestos once used to make kent cigarette filters

N years old and former chairman of consolidated gold fields plc was named a nonexecutive director
"""

sequence = word_indexing.function(sentences)
sequence

Sentence is empty. Skipping...
Sentence is empty. Skipping...
Sentence is empty. Skipping...


array([[37, 62, 44, 63, 64, 65, 66, 67, 68, 69, 70,  0,  0,  0,  0,  0],
       [29, 30, 31, 50, 51, 43, 44, 52, 53, 54, 55, 56, 57, 37, 38, 39]])

## Target, context pairs

For each word in the setence ```(of, asbestos, ... , kent)``` excludnig the ends of the sentence, create ```(target, context)``` as:

```
[
  [of, a, form, asbestos, once],              # target is 'of', context is (a, form, asbestos, once)
  ['asbestos', 'form', 'of', 'once', 'used'],
  ['once', 'of', 'asbestos', 'used', 'to'],
  ...
]
```

### Format of the (event, context) pairs

* **E** is the target event indices
* **C** is the context indices

<img src="image/event_context_format.png" align="left"/>

In [19]:
event_context_pairs = event_context.function(sequence)
print(
    f"Event context pairs. Shape %s, Target event size %s, Window size %s." % 
    (event_context_pairs.shape, event_context.event_size, event_context.window_size)
)
event_context_pairs[:5]

Event context pairs. Shape (21, 5), Target event size 1, Window size 5.


array([[44, 37, 62, 63, 64],
       [63, 62, 44, 64, 65],
       [64, 44, 63, 65, 66],
       [65, 63, 64, 66, 67],
       [66, 64, 65, 67, 68]], dtype=int32)

### Target, context pairs in textual words

In [20]:
word_indexing.sequence_to_sentence(event_context_pairs[:5])

[['of', 'a', 'form', 'asbestos', 'once'],
 ['asbestos', 'form', 'of', 'once', 'used'],
 ['once', 'of', 'asbestos', 'used', 'to'],
 ['used', 'asbestos', 'once', 'to', 'make'],
 ['to', 'once', 'used', 'make', 'kent']]

---
# Word Embedding

Embedding is to train the model to group similar events in a close proximity in the event vector space. If two events e.g. 'pencil' and 'pen' are similar concepts, then their event vectors resides in a close distance in the event space. 

* [Thought Vectors](https://wiki.pathmind.com/thought-vectors)

## Training process

1. Calculate ```Bc```, the BoW (Bag of Words) from context event vectors.
2. Calculate ```Be```,  the BoW (Bag of Words) from target event vectors.
3. The dot product ```Ye = dot(Bc, Be)``` is given the label 1 to get them closer.
4. For each negative sample ```Ws(s)```, the dot product with ```Ys = dot(Be, Ws(s)``` is given the label 0 to get them apart. 
5. ```np.c_[Ye, Ys]``` is fowarded to the logistic log loss layer.

<img src="image/word2vec_backprop_Be.png" align="left"/>


In [21]:
%autoreload 2
from layer import (
    Embedding
)

In [22]:
embedding: Embedding = Embedding(
    name="embedding",
    num_nodes=WINDOW_SIZE,
    target_size=TARGET_SIZE,
    context_size=CONTEXT_SIZE,
    negative_sample_size=SAMPLE_SIZE,
    event_vector_size=VECTOR_SIZE,
    dictionary=word_indexing
)

### Scores ```np.c_[Ye, Ys]``` from the Embedding

The 0th column is the scores for ```dot(Bc, Be``` for positive labels. The rest are the scores for ```dot(Bc, Ws)``` for negative labels.

In [23]:
scores = embedding.function(event_context_pairs)
print(f"Scores:\n{scores[:5]}\n")
print(f"Scores for dot(Bc, Be): \n{scores[:5, :1]}")

Scores:
[[28.156822 29.552954 30.100132 34.552273 27.803158 28.692389]
 [25.148438 34.60143  28.760263 31.664251 33.420506 30.077932]
 [31.103136 26.138912 23.508234 28.221443 28.13847  26.96113 ]
 [29.687222 30.451935 30.893887 24.217983 27.653366 33.007046]
 [30.101013 30.144602 27.305859 34.857613 32.30297  29.19872 ]]

Scores for dot(Bc, Be): 
[[28.156822]
 [25.148438]
 [31.103136]
 [29.687222]
 [30.101013]]


### Prediction from the Embedding

After training, the prediction is expected to provide a vector which is in the close proximity to the target.

In [24]:
context_indices = event_context_pairs[
    ::,    # Event(1)/Context(4) pairs
    :1     # Context
]
context = context_indices[0]
word_indexing.sequence_to_sentence(context)

print(embedding.predict(context))

[0.96357924 0.5271716  0.2576603  0.58239764 0.27313867 0.9599152  0.65803885 0.8845882  0.35819176 0.09486984 0.22594991 0.87400544 0.4738174  0.65891886 0.62385386 0.11334255 0.89815503 0.31788802 0.50352126 0.15580328 0.46780875 0.2534049  0.06589965 0.7170765  0.19394475 0.84001774 0.6213395  0.6049851  0.1314408  0.4599199 ]


---
# Logistic Log Loss

Train the model to get:
1. BoW of the context event vectors close to the target event vector. Label 1
2. BoW of the context event vectors away from each of the negative sample event vectors Label 0.

This is a binary logistic classification, hence use Logistic Log Loss as the network objective function.

In [25]:
from common.function import (
    sigmoid_cross_entropy_log_loss,
    sigmoid
)
from layer.objective import (
    CrossEntropyLogLoss
)

In [26]:
loss = CrossEntropyLogLoss(
    name="loss",
    num_nodes=1,  # Logistic log loss
    log_loss_function=sigmoid_cross_entropy_log_loss
)

---
# Adapter

The logistic log loss layer expects the input of shape ```(N,M=1)```, however Embedding outputs ```(N,(1+SL)``` where ```SL``` is SAMPLE_SIZE. The ```Adapter``` layer bridges between Embedding and the Loss layers.


In [27]:
from layer.adapter import (
    Adapter
)

In [28]:
adapter_function = embedding.adapt_function_to_logistic_log_loss(loss=loss)
adapter_gradient = embedding.adapt_gradient_to_logistic_log_loss()

adapter: Adapter = Adapter(
    name="adapter",
    num_nodes=1,    # Number of output M=1 
    function=adapter_function,
    gradient=adapter_gradient
)

---

---
# Network

In [29]:
from network.sequential import (
    SequentialNetwork
)

In [30]:
network = SequentialNetwork(
    name="word2vec",
    num_nodes=1,
    inference_layers=[
        word_indexing,
        event_context,
        embedding,
        adapter
    ],
    objective_layers=[
        loss
    ]
)

In [59]:
import string
lines = """

  tako
ika
bin 
   uni  
"""
lines.replace('\n', '').replace(' ', '')

'takoikabinuni'

In [63]:
def train():
    stream = fileio.Function.file_line_stream(path_to_ptb)
    try:
        while True:
            lines = fileio.Function.take(NUM_SENTENCES, stream)
            if len(''.join(lines).replace('\n', '').replace(' ', '')) > 0:
                sentences = np.array(lines)
                yield sentences
    finally:
        stream.close()

# Restore the state if exists.
STATE_FILE = "wor2vec_embedding.pkl"
if fileio.Function.is_file(STATE_FILE):
    embedding.load(STATE_FILE)

# Continue training
profiler = cProfile.Profile()
profiler.enable()

NUM_SENTENCES = 100
MAX_ITERATIONS = 1000

epochs = 0
trainer = train()
for i in range(MAX_ITERATIONS):
    try:
        network.train(X=next(trainer), T=np.array([0]))
        if i % 25 == 0:
            print(f"{i:05d}: Loss: {network.history[-1]:15f}")
        if i % 100 == 0:
            embedding.save(STATE_FILE)
    except StopIteration as e:
        # Next epoch
        epochs += 1
        trainer = train()
        pass
    except Exception as e:
        print("Unexpected error {0}".format(str(e)))
        trainer.close()
    finally:
        embedding.save(STATE_FILE)

profiler.disable()
profiler.print_stats(sort="cumtime")

00000: Loss:        6.315559
00025: Loss:        6.845446
00050: Loss:        6.988423
00075: Loss:        6.146124
00100: Loss:        6.541249
00125: Loss:        6.045611
00150: Loss:        6.756516
00175: Loss:        6.071086


KeyboardInterrupt: 

In [33]:
embedding.save("wor2vec_embedding.pkl")